In [1]:
sc

In [2]:
data = sc.textFile('data/sales_segments.csv.gz')

In [3]:
data.take(1)

[u'rloc^departure_date_seg^booking_date^freq_flyer_flag^creator_office_id^creator_iata_number^creator_gds_code^creator_city_code^creator_lat^creator_lon^creator_country_code^creator_continent_code^creator_channel_code^booking_status_code^cabin_code^booking_class_code^pos_office_id^pos_iata_number^pos_gds_code^pos_city_code^pos_lat^pos_lon^pos_country_code^pos_continent_code^pos_channel_code^board_point_seg^board_lat^board_lon^board_country_code^board_continent_code^off_point_seg^off_lat^off_lon^off_country_code^off_continent_code^num_legs^distance_seg^route^network^airline_code^flight_number^flight_type^revenue_source^estimated_flag^usage_board_point_seg^usage_off_point_seg^usage_flight_number^usage_date^non_currency_type^bookings_seg^rpk_seg^revenue_amount_seg^currency_code^fuel_surcharge_amount_seg^fuel_surcharge_currency_code^emd_count^emd_amount^emd_currency_code^total_outliers']

In [4]:
#para eliminar la primera fila
header = data.first()

In [5]:
lines = data.filter(lambda line: line != header)

In [6]:
lines.take(1)

[u'59GS9Y^2013-12-21^2013-09-05^false^RUNHS3103^66200201^1A^RUN^-20.90083^55.53715^RE^Africa^Travel Agent^HK^Y^N^RUNHS3103^66200201^1A^RUN^-20.90083^55.53715^RE^Africa^Travel Agent^ZYR^50.8348^4.33653^BE^Europe^CDG^49.01278^2.55^FR^Europe^0^239.573363253866^CDG-ZYR^Europe^UU^5901^I^LIFT^false^ZYR^CDG^5901^2013-12-21^^1^239^33.56^EUR^7.30^EUR^0^0.00^^7.3']

In [7]:
lines.first().split('^')

[u'59GS9Y',
 u'2013-12-21',
 u'2013-09-05',
 u'false',
 u'RUNHS3103',
 u'66200201',
 u'1A',
 u'RUN',
 u'-20.90083',
 u'55.53715',
 u'RE',
 u'Africa',
 u'Travel Agent',
 u'HK',
 u'Y',
 u'N',
 u'RUNHS3103',
 u'66200201',
 u'1A',
 u'RUN',
 u'-20.90083',
 u'55.53715',
 u'RE',
 u'Africa',
 u'Travel Agent',
 u'ZYR',
 u'50.8348',
 u'4.33653',
 u'BE',
 u'Europe',
 u'CDG',
 u'49.01278',
 u'2.55',
 u'FR',
 u'Europe',
 u'0',
 u'239.573363253866',
 u'CDG-ZYR',
 u'Europe',
 u'UU',
 u'5901',
 u'I',
 u'LIFT',
 u'false',
 u'ZYR',
 u'CDG',
 u'5901',
 u'2013-12-21',
 u'',
 u'1',
 u'239',
 u'33.56',
 u'EUR',
 u'7.30',
 u'EUR',
 u'0',
 u'0.00',
 u'',
 u'7.3']

In [8]:
#variables con las que vamo a trabajar en el clustering
fields = ['bookings_seg', 'revenue_amount_seg', 'fuel_surcharge_amount_seg']

In [9]:
header.split('^').index('bookings_seg')

49

In [10]:
#iterar sobre fields para obtener los elementos correspondientes
[header.split('^').index(field) for field in fields]

[49, 51, 53]

In [16]:
def average_fare_tax(line):
    e = line.split('^')
    bookings = float(e[49])
    rev = float(e[51])
    tax = float(e[53])
    
    av_rev = rev/bookings
    av_tax = tax/bookings
    
    return (av_rev, av_tax)

In [17]:
average_fare_tax(lines.first())

(33.56, 7.3)

In [18]:
simple = lines.map(average_fare_tax)

In [19]:
simple.take(3)

[(33.56, 7.3), (61.2, 25.5), (71.86842105263158, 12.31578947368421)]

In [21]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [22]:
clusters = KMeans.train(simple, 10)

In [23]:
clusters.k

10

In [24]:
clusters.centers

[array([ 431.19140489,  184.92581272]),
 array([  8.77500428e+05,   3.03020000e+01]),
 array([  6.15206951e+05,   2.97312500e+01]),
 array([  1.08940744e+05,   2.14350000e+01]),
 array([ 109.20412744,   63.82547158]),
 array([  1.39215574e+06,   3.27400000e+01]),
 array([  4.43932834e+04,   3.30665278e+01]),
 array([  3.72063199e+05,   1.03505455e+02]),
 array([ 1569.05656347,   208.88342533]),
 array([  2.62467806e+05,   7.43850000e+01])]

In [25]:
clusters.predict((33.56, 7.3))

4

In [35]:
bycluster = simple.map(lambda point: (clusters.predict(point), point))

In [30]:
# devolvemos como clave el cluster y luego los valores av_rev y av_tax
bycluster.take(3)

[(4, (33.56, 7.3)),
 (4, (61.2, 25.5)),
 (4, (71.86842105263158, 12.31578947368421))]

In [ ]:
# versión no óptima debido al posible tamaño de cada grupo
# bycluster.groupByKey().take(3)

[(0, <pyspark.resultiterable.ResultIterable at 0x7f2f0d579cd0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f2f0d65bc50>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7f2f0d5cbcd0>)]

In [ ]:
def calc_avg_reduce()

In [33]:
means = bycluster.reduceByKey(calc_avg_reduce)

NameError: name 'calc_avg_reduce' is not defined

In [34]:
bycluster_whithones = simple.map(lambda (x,y): (clusters.predict((x,y)), (x,y,1)))

In [37]:
bycluster_whithones.cache()

PythonRDD[64] at RDD at PythonRDD.scala:43

In [38]:
bycluster_whithones.take(3)

[(4, (33.56, 7.3, 1)),
 (4, (61.2, 25.5, 1)),
 (4, (71.86842105263158, 12.31578947368421, 1))]

In [45]:
def calc_avg_reduce(acc,p):
    (rev_p, tax_p, c) = p
    (rev_ac, tax_ac, c_ac) = acc
    result = (rev_p + rev_ac, tax_p + tax_ac, c + c_ac)
    return result

In [46]:
bycluster_whithones.reduceByKey(calc_avg_reduce).take(3)

[(0, (152480491.74609438, 65394575.44836157, 353626)),
 (1, (8775004.28, 303.02, 10)),
 (2, (4921655.609999999, 237.84999999999997, 8))]

In [47]:
avs = bycluster_whithones.reduceByKey(calc_avg_reduce)

In [48]:
avs.take(3)

[(0, (152480491.74609438, 65394575.44836157, 353626)),
 (1, (8775004.28, 303.02, 10)),
 (2, (4921655.609999999, 237.84999999999997, 8))]